In [1]:

using Revise
includet("../general_modules/channels.jl")
include("../general_modules/mesh.jl")
using .channels
using .mesh
using JSON

In [2]:
data = JSON.parsefile("test.json")
Systems_params= data["Systems"]
mesh_params= data["Mesh"]
fermion=true
α= α3b(fermion,0.5,0.5,1,Systems_params["lmax"],Systems_params["lmin"],Systems_params["λmax"],Systems_params["λmin"],Systems_params["s1"],Systems_params["s2"],Systems_params["s3"],Systems_params["t1"],Systems_params["t2"],Systems_params["t3"],Systems_params["MT"])

grid= initialmesh(mesh_params["nθ"],mesh_params["nx"],mesh_params["ny"],Float64(mesh_params["xmax"]),Float64(mesh_params["ymax"]),Float64(mesh_params["alpha"]))

For J=0.5 T=0.5 parity=1 Number of channels: 2
scaling factor for x: 0.18859695545615252


Main.mesh.meshset(12, 30, 30, [-0.9815606342467192, -0.9041172563704748, -0.7699026741943047, -0.5873179542866175, -0.3678314989981802, -0.1252334085114689, 0.1252334085114689, 0.3678314989981802, 0.5873179542866175, 0.7699026741943047, 0.9041172563704748, 0.9815606342467192], [0.04717533638651175, 0.10693932599531826, 0.16007832854334625, 0.20316742672306587, 0.23349253653835492, 0.24914704581340288, 0.24914704581340288, 0.23349253653835492, 0.20316742672306587, 0.16007832854334625, 0.10693932599531826, 0.04717533638651175], [0.02233755499529331, 0.07493852941322005, 0.15777170988775746, 0.2710468592921538, 0.41506220089174284, 0.5902044222346922, 0.7969539970971387, 1.0358926739011418, 1.3077130639006518, 1.6132306695796537  …  7.627531701911872, 8.486257300100586, 9.419116142271335, 10.436398422075927, 11.551987684483082, 12.78547931418975, 14.16630994957712, 15.74295182755926, 17.608089697752572, 20.0], [0.03752824519482659, 0.06768997721952366, 0.09801166074965893, 0.1285883051482

In [ ]:
for i in 1:mesh_params["nx"]
    println(grid.xi[i], "   ", grid.dxi[i])
end

In [ ]:
using Revise
Revise.revise()
includet("Gcoefficient.jl")
using .Gcoefficient

initialY(λmax, lmax, nθ, nx, ny, cosθi, xi, yi, P::Char)
return Yλαout, Ylαin, Yλαin

Yλin[iθ, iy, ix, nch, perm_index]

In [ ]:
Gαα=computeGcoefficient(α, grid)

In [ ]:
using Revise
includet("twobody.jl")
using .twobodybound

In [ ]:
potname="MT"
bound_states=bound2b(grid, potname)

In [ ]:
using Revise
includet("spline.jl")
using .spline

In [ ]:
"""
Test the cubic Hermitian spline interpolation with a polynomial function.
"""
# Import Printf for formatting
using Printf
using Plots
function test_cubherm_interpolation()
    # Define the polynomial function f(x) = 5x^5 + 4x^4 + 3x^3 + 2x^2 + x
    f(x) = 5*x^5 + 4*x^4 + 3*x^3 + 2*x^2 + x
    
    # Create a set of old grid points (coarse grid)
    xold = collect(range(-1.0, 1.0, length=15))
    
    # Evaluate the function at the old grid points
    yold = f.(xold)
    
    # println("Original grid points:")
    # for i in 1:length(xold)
    #     println("x = $(xold[i]), f(x) = $(yold[i])")
    # end
    # println()
    
    # Create a set of test points (fine grid) where we want to interpolate
    xtest = collect(range(-0.95, 0.95, length=20))
    
    # Compute the exact values
    yexact = f.(xtest)
    
    # Interpolate using our cubic Hermitian spline
    yinterp = Float64[]
    for x in xtest
        push!(yinterp, cubherm_interp_point(xold, yold, x))
    end
    
    # Compute errors
    errors = abs.(yinterp - yexact)
    max_error = maximum(errors)
    avg_error = sum(errors) / length(errors)
    
    println("Interpolation results and errors:")
    println("   x       Exact        Interpolated       Error")
    println("------------------------------------------------------")
    for i in 1:length(xtest)
        # Use simple string formatting instead of @printf/@sprintf
        x_str = @sprintf("%.4f", xtest[i])
        exact_str = @sprintf("%.6f", yexact[i])
        interp_str = @sprintf("%.6f", yinterp[i])
        error_str = @sprintf("%.6e", errors[i])
        
        println("$(lpad(x_str, 7))  $(lpad(exact_str, 12))  $(lpad(interp_str, 12))  $(lpad(error_str, 12))")
    end
    
    println("\nMaximum error: $max_error")
    println("Average error: $avg_error")
    
    # Optional: Plot the results if Plots.jl is available
    try
        
        
        p = plot(xold, yold, seriestype=:scatter, 
             label="Original Points", markersize=6, legend=:topleft)
        
        plot!(p, xtest, yexact, 
              label="Exact Function", linewidth=2)
        
        plot!(p, xtest, yinterp, 
              label="Interpolated", linewidth=2, linestyle=:dash)
        
        # Plot error
        p_error = plot(xtest, errors, 
                   label="Interpolation Error", linewidth=2,
                   xlabel="x", ylabel="Error", title="Interpolation Error")
        
        # Display both plots
        plot(p, p_error, layout=(2,1), size=(800, 600),
             title=["Cubic Hermitian Spline Interpolation" ""])
        
        savefig("interpolation_test.png")
        println("\nPlot saved as 'interpolation_test.png'")
    catch
        println("\nPlotting skipped (Plots.jl not available)")
    end
    
    return max_error, avg_error
end

In [ ]:
using Printf 
test_cubherm_interpolation()

In [3]:
using Revise
includet("matrices.jl")
using .matrices


In [ ]:
Rxy=Rxy_matrix(α, grid)
println("Rxy matrix:", Rxy[30,144])

In [3]:
using Revise
includet("threebodybound.jl")
using .threebodybound

In [4]:
ThreeBody_Bound(α, grid)

Number of bound states: 4
Bound state energies: ComplexF64[-9.333888242094295 - 3.411545631718321e-14im, -7.1350235218777645 + 4.1551642286712397e-14im, -4.645634472708434 - 2.9585992890054896e-14im, -0.8655240300723803 + 3.929176881579563e-14im]


4-element Vector{Any}:
 (-9.333888242094295 - 3.411545631718321e-14im, ComplexF64[1.1106371898053781e-7 - 4.665298654799076e-22im, 3.515137725668958e-7 - 2.4914680050097194e-21im, -5.610480851041702e-7 - 6.815968247436948e-21im, -6.6776212856079455e-6 - 2.329340604949326e-21im, -2.4976218447725498e-5 + 3.3881317890172014e-21im, -5.927691029897667e-5 - 1.2281977735187355e-20im, -0.00010349473790929702 - 1.3552527156068805e-19im, -0.00014126557965582717 - 3.5405977195229754e-19im, -0.00015675015291558746 - 5.183841637196318e-19im, -0.0001468914252912197 - 4.438452643612534e-19im  …  -0.0 + 0.0im, -0.0 + 0.0im, -0.0 + 0.0im, -0.0 + 0.0im, -0.0 + 0.0im, -0.0 + 0.0im, -0.0 + 0.0im, -0.0 + 0.0im, -0.0 + 0.0im, -0.0 + 0.0im])
 (-7.1350235218777645 + 4.1551642286712397e-14im, ComplexF64[-5.896574406211195e-8 - 1.826414792517085e-21im, -1.059185987774696e-7 + 1.0521737391674512e-21im, 1.6298652905029477e-7 + 4.622947007436361e-20im, 2.606215660379838e-6 + 2.90532300908225e-19im, 9.1861218960051

In [ ]:
T_matrix(α,grid) 

In [4]:
V_matrix(α, grid, "MT")

1800×1800 Kronecker.KroneckerProduct{Float64, Matrix{Float64}, Matrix{Float64}}:
 50479.3      0.0      0.0      0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
     0.0  50479.3      0.0      0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
     0.0      0.0  50479.3      0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
     0.0      0.0      0.0  50479.3     0.0  0.0  0.0  0.0  0.0  0.0  0.0
     0.0      0.0      0.0      0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
     0.0      0.0      0.0      0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
     0.0      0.0      0.0      0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
     0.0      0.0      0.0      0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
     0.0      0.0      0.0      0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
     0.0      0.0      0.0      0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
     ⋮                               ⋱            ⋮                   
     0.0      0.0      0.0      0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
     0.0      0.0      0.0      0.